### Synthetic Data Backtest

Most of the functions below can be found under:

* Tool/backtest 
* Sample_data/make_data

A quick slideshow: [Seven Sins of Backtesting](https://newyork.qwafafew.org/wp-content/uploads/sites/4/2015/10/Luo_20150128.pdf)

The below is a code snippet for trading rules optimization.

The origin exercise required to use real-HFT data to deduce a certain outcome.. which I don't have.

Hence, I will just replace this exercise with multiprocessing methods (Parallelization!).

If you are keen on generating synthetic data for your research, copy the code snippets [Generate synthetic raw data](https://gist.github.com/boyboi86/5e00faf48f60abfdbe838fbdee269471) in my gist.

Contact: boyboi86@gmail.com

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import research as rs
from random import gauss
from itertools import product

%matplotlib inline

Num of CPU core:  4
Machine info:  Windows-10-10.0.18362-SP0
Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Numpy 1.18.5
Pandas 1.0.4


In [2]:
dict0 = {'a':['1', '2'], 'b':['+', '*'], 'c':['!','A']}
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print({'a':a, 'b':b, 'c':c})

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

The above is scalar operation. Notice loop after loop after loop.

The below is vectorization operation, which is what we will do for Monte-Carlos.

The result is actually the same.

In [3]:
jobs = (dict(zip(dict0, i )) for i in product(*dict0.values()))
for i in jobs: print(i)

{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': 'A'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': 'A'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': 'A'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': 'A'}


**Note**

Given the parameters below. It should look like the below.

In [4]:
forecast = [10,5,0,-5,-10]
half_life = [5,10,25,50,100]
loss_range = profit_range = np.linspace(.5, 10, 20)
sigma = [1]
max_period = half_life[-1]
n_run = 100

coeff = dict({'forecast': forecast,
              'half_life': half_life,
              'sigma':sigma,
              'profit_range': profit_range,
              'loss_range':loss_range})
jobs = list(dict(zip(coeff, idx)) for idx in product(*coeff.values()))

print(len(jobs)) #forecast * half_life * loss * profit * sigma
out = pd.DataFrame(jobs)
out

10000


,forecast,half_life,sigma,profit_range,loss_range
0,10,5,1,0.5,0.5
1,10,5,1,0.5,1.0
2,10,5,1,0.5,1.5
3,10,5,1,0.5,2.0
4,10,5,1,0.5,2.5
...,...,...,...,...,...
9995,-10,100,1,10.0,8.0
9996,-10,100,1,10.0,8.5
9997,-10,100,1,10.0,9.0
9998,-10,100,1,10.0,9.5


**Note**

The below is the end product for a non-parallized Monte-Carlos Simulation, for optimizied Trading Rule Algorithm.

Primarily we will use it to create scenarios for our models to be backtest with.

There is another algorithm which MLfinlab has implemented, you may wish to read that as well.

[Backtesting by Campbell and Yan [2015]](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2345489)

In [5]:
def _opt_tr(coeff: dict = None,
            n_run: float = 100,
            max_period: int = 100,
            seed: int = 0):
    
    phi, output1 = 2 ** (-1/ coeff['half_life']), []
    for iter_ in range(int(n_run)):
        p, hold_period, count = seed, 0, 0
        while True:
            p = (1 - phi) * coeff['forecast'] + phi * p + coeff['sigma'] * gauss(0,1)
            cp = p - seed
            hold_period += 1
            if coeff['profit_range'] < cp or -coeff['loss_range'] > cp or hold_period > max_period:
                output1.append(cp)
                break
    mean, std = np.mean(output1), np.std(output1)
    return {'profit': coeff['profit_range'], 'loss': coeff['loss_range'], 'mean': mean, 'std': std, 'sharpe': mean/std}

def opt_tr(profit_range: list = np.linspace(.5, 10, 20),
          loss_range: list = np.linspace(.5, 10, 20),
          sigma: list = [1],
          n_run: float = 100,
          max_period: int = 100,
          forecast: list = [10,5,0,-5,-10], 
          half_life: list = [5,10,25,50,100],
          num_threads: int = 1):
    
    if profit_range is None:
        profit_range = np.linspace(0,10,21)
        
    if loss_range is None:
        loss_range = np.linspace(0,10,21)
        
    _coeff_list = []
    _coeff = dict({'forecast': forecast,
                   'half_life': half_life,
                   'sigma':sigma,
                   'profit_range': profit_range,
                   'loss_range':loss_range})
    
    _coeff = list(dict(zip(_coeff, idx )) for idx in product(*_coeff.values()))
    out = []
    for coeff in _coeff:
        jobs = [{'func': _opt_tr,
                   'coeff': coeff,
                   'n_run': n_run, 
                   'max_period': max_period}]
        
        if num_threads == 1:
            out_ = rs.process_jobs_(jobs)
            out.append(out_)
        else:
            out_ = rs.process_jobs(jobs, num_threads = num_threads)
            out.append(out_)

    return out

df = opt_tr(profit_range = profit_range,
              loss_range = loss_range,
              sigma = sigma,
              n_run = n_run,
              max_period = max_period,
              forecast = forecast, 
              half_life = half_life,
              num_threads = 1)

2020-06-17 14:41:28.739905 done after 0.00 mins
2020-06-17 14:41:28.741900 done after 0.00 mins
2020-06-17 14:41:28.742898 done after 0.00 mins
2020-06-17 14:41:28.744894 done after 0.00 mins
2020-06-17 14:41:28.745889 done after 0.00 mins
2020-06-17 14:41:28.746887 done after 0.00 mins
2020-06-17 14:41:28.747883 done after 0.00 mins
2020-06-17 14:41:28.749879 done after 0.00 mins
2020-06-17 14:41:28.750875 done after 0.00 mins
2020-06-17 14:41:28.751873 done after 0.00 mins
2020-06-17 14:41:28.752870 done after 0.00 mins
2020-06-17 14:41:28.754865 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.756001 done after 0.00 mins
2020-06-17 14:41:28.764031 done after 0.00 mins
2020-06-17 14:41:28.764031 done after 0.00 mins
2020-06-17 14:41:28.766057 done after 0.

2020-06-17 14:41:29.527935 done after 0.00 mins
2020-06-17 14:41:29.535974 done after 0.00 mins
2020-06-17 14:41:29.538002 done after 0.00 mins
2020-06-17 14:41:29.538002 done after 0.00 mins
2020-06-17 14:41:29.546040 done after 0.00 mins
2020-06-17 14:41:29.548069 done after 0.00 mins
2020-06-17 14:41:29.548069 done after 0.00 mins
2020-06-17 14:41:29.588342 done after 0.00 mins
2020-06-17 14:41:29.588342 done after 0.00 mins
2020-06-17 14:41:29.595040 done after 0.00 mins
2020-06-17 14:41:29.598032 done after 0.00 mins
2020-06-17 14:41:29.601024 done after 0.00 mins
2020-06-17 14:41:29.604016 done after 0.00 mins
2020-06-17 14:41:29.608006 done after 0.00 mins
2020-06-17 14:41:29.612994 done after 0.00 mins
2020-06-17 14:41:29.616984 done after 0.00 mins
2020-06-17 14:41:29.622237 done after 0.00 mins
2020-06-17 14:41:29.659086 done after 0.00 mins
2020-06-17 14:41:29.667120 done after 0.00 mins
2020-06-17 14:41:29.669153 done after 0.00 mins
2020-06-17 14:41:29.669153 done after 0.

2020-06-17 14:41:31.586821 done after 0.00 mins
2020-06-17 14:41:31.592380 done after 0.00 mins
2020-06-17 14:41:31.602464 done after 0.00 mins
2020-06-17 14:41:31.612528 done after 0.00 mins
2020-06-17 14:41:31.660900 done after 0.00 mins
2020-06-17 14:41:31.671104 done after 0.00 mins
2020-06-17 14:41:31.673240 done after 0.00 mins
2020-06-17 14:41:31.683564 done after 0.00 mins
2020-06-17 14:41:31.724143 done after 0.00 mins
2020-06-17 14:41:31.732265 done after 0.00 mins
2020-06-17 14:41:31.742368 done after 0.00 mins
2020-06-17 14:41:31.752433 done after 0.00 mins
2020-06-17 14:41:31.772573 done after 0.00 mins
2020-06-17 14:41:31.782637 done after 0.00 mins
2020-06-17 14:41:31.792912 done after 0.00 mins
2020-06-17 14:41:31.836663 done after 0.00 mins
2020-06-17 14:41:31.846882 done after 0.00 mins
2020-06-17 14:41:31.856856 done after 0.00 mins
2020-06-17 14:41:31.864834 done after 0.00 mins
2020-06-17 14:41:31.886906 done after 0.00 mins
2020-06-17 14:41:31.896966 done after 0.

2020-06-17 14:41:33.201956 done after 0.00 mins
2020-06-17 14:41:33.201956 done after 0.00 mins
2020-06-17 14:41:33.212064 done after 0.00 mins
2020-06-17 14:41:33.212064 done after 0.00 mins
2020-06-17 14:41:33.222134 done after 0.00 mins
2020-06-17 14:41:33.222134 done after 0.00 mins
2020-06-17 14:41:33.272963 done after 0.00 mins
2020-06-17 14:41:33.272963 done after 0.00 mins
2020-06-17 14:41:33.281002 done after 0.00 mins
2020-06-17 14:41:33.283030 done after 0.00 mins
2020-06-17 14:41:33.290958 done after 0.00 mins
2020-06-17 14:41:33.296942 done after 0.00 mins
2020-06-17 14:41:33.301929 done after 0.00 mins
2020-06-17 14:41:33.340935 done after 0.00 mins
2020-06-17 14:41:33.342961 done after 0.00 mins
2020-06-17 14:41:33.350991 done after 0.00 mins
2020-06-17 14:41:33.353042 done after 0.00 mins
2020-06-17 14:41:33.353042 done after 0.00 mins
2020-06-17 14:41:33.363124 done after 0.00 mins
2020-06-17 14:41:33.363124 done after 0.00 mins
2020-06-17 14:41:33.373188 done after 0.

2020-06-17 14:41:35.946711 done after 0.00 mins
2020-06-17 14:41:35.956779 done after 0.00 mins
2020-06-17 14:41:35.966887 done after 0.00 mins
2020-06-17 14:41:35.976949 done after 0.00 mins
2020-06-17 14:41:36.027579 done after 0.00 mins
2020-06-17 14:41:36.045709 done after 0.00 mins
2020-06-17 14:41:36.093386 done after 0.00 mins
2020-06-17 14:41:36.109348 done after 0.00 mins
2020-06-17 14:41:36.123306 done after 0.00 mins
2020-06-17 14:41:36.154895 done after 0.00 mins
2020-06-17 14:41:36.166491 done after 0.00 mins
2020-06-17 14:41:36.178635 done after 0.00 mins
2020-06-17 14:41:36.240011 done after 0.00 mins
2020-06-17 14:41:36.250093 done after 0.00 mins
2020-06-17 14:41:36.260267 done after 0.00 mins
2020-06-17 14:41:36.320986 done after 0.00 mins
2020-06-17 14:41:36.331490 done after 0.00 mins
2020-06-17 14:41:36.349834 done after 0.00 mins
2020-06-17 14:41:36.408013 done after 0.00 mins
2020-06-17 14:41:36.421983 done after 0.00 mins
2020-06-17 14:41:36.431948 done after 0.

...

2020-06-17 14:46:24.306418 done after 0.00 mins
2020-06-17 14:46:24.316489 done after 0.00 mins
2020-06-17 14:46:24.324578 done after 0.00 mins
2020-06-17 14:46:24.373306 done after 0.00 mins
2020-06-17 14:46:24.384274 done after 0.00 mins
2020-06-17 14:46:24.395253 done after 0.00 mins
2020-06-17 14:46:24.438642 done after 0.00 mins
2020-06-17 14:46:24.457886 done after 0.00 mins
2020-06-17 14:46:24.468037 done after 0.00 mins
2020-06-17 14:46:24.516483 done after 0.00 mins
2020-06-17 14:46:24.518640 done after 0.00 mins
2020-06-17 14:46:24.518640 done after 0.00 mins
2020-06-17 14:46:24.518640 done after 0.00 mins
2020-06-17 14:46:24.528806 done after 0.00 mins
2020-06-17 14:46:24.528806 done after 0.00 mins
2020-06-17 14:46:24.567094 done after 0.00 mins
2020-06-17 14:46:24.569245 done after 0.00 mins
2020-06-17 14:46:24.579352 done after 0.00 mins
2020-06-17 14:46:24.593381 done after 0.00 mins
2020-06-17 14:46:24.637625 done after 0.00 mins
2020-06-17 14:46:24.647990 done after 0.

2020-06-17 14:46:29.133603 done after 0.00 mins
2020-06-17 14:46:29.153764 done after 0.00 mins
2020-06-17 14:46:29.214344 done after 0.00 mins
2020-06-17 14:46:29.305100 done after 0.00 mins
2020-06-17 14:46:29.325275 done after 0.00 mins
2020-06-17 14:46:29.441466 done after 0.00 mins
2020-06-17 14:46:29.496318 done after 0.00 mins
2020-06-17 14:46:29.559542 done after 0.00 mins
2020-06-17 14:46:29.628006 done after 0.00 mins
2020-06-17 14:46:29.698924 done after 0.00 mins
2020-06-17 14:46:29.767523 done after 0.00 mins
2020-06-17 14:46:29.772547 done after 0.00 mins
2020-06-17 14:46:29.776537 done after 0.00 mins
2020-06-17 14:46:29.782521 done after 0.00 mins
2020-06-17 14:46:29.791496 done after 0.00 mins
2020-06-17 14:46:29.800471 done after 0.00 mins
2020-06-17 14:46:29.840006 done after 0.00 mins
2020-06-17 14:46:29.850452 done after 0.00 mins
2020-06-17 14:46:29.910154 done after 0.00 mins
2020-06-17 14:46:29.927110 done after 0.00 mins
2020-06-17 14:46:29.972794 done after 0.

2020-06-17 14:46:37.517759 done after 0.00 mins
2020-06-17 14:46:37.527863 done after 0.00 mins
2020-06-17 14:46:37.527863 done after 0.00 mins
2020-06-17 14:46:37.527863 done after 0.00 mins
2020-06-17 14:46:37.565170 done after 0.00 mins
2020-06-17 14:46:37.569158 done after 0.00 mins
2020-06-17 14:46:37.572150 done after 0.00 mins
2020-06-17 14:46:37.576140 done after 0.00 mins
2020-06-17 14:46:37.580129 done after 0.00 mins
2020-06-17 14:46:37.584118 done after 0.00 mins
2020-06-17 14:46:37.588095 done after 0.00 mins
2020-06-17 14:46:37.596128 done after 0.00 mins
2020-06-17 14:46:37.633520 done after 0.00 mins
2020-06-17 14:46:37.639044 done after 0.00 mins
2020-06-17 14:46:37.649110 done after 0.00 mins
2020-06-17 14:46:37.659182 done after 0.00 mins
2020-06-17 14:46:37.659182 done after 0.00 mins
2020-06-17 14:46:37.679325 done after 0.00 mins
2020-06-17 14:46:37.689398 done after 0.00 mins
2020-06-17 14:46:37.699497 done after 0.00 mins
2020-06-17 14:46:37.709564 done after 0.

2020-06-17 14:46:41.712874 done after 0.00 mins
2020-06-17 14:46:41.769662 done after 0.00 mins
2020-06-17 14:46:41.797886 done after 0.00 mins
2020-06-17 14:46:41.854663 done after 0.00 mins
2020-06-17 14:46:41.910664 done after 0.00 mins
2020-06-17 14:46:41.961403 done after 0.00 mins
2020-06-17 14:46:42.022263 done after 0.00 mins
2020-06-17 14:46:42.097164 done after 0.00 mins
2020-06-17 14:46:42.101154 done after 0.00 mins
2020-06-17 14:46:42.107137 done after 0.00 mins
2020-06-17 14:46:42.113120 done after 0.00 mins
2020-06-17 14:46:42.122098 done after 0.00 mins
2020-06-17 14:46:42.130078 done after 0.00 mins
2020-06-17 14:46:42.183932 done after 0.00 mins
2020-06-17 14:46:42.197894 done after 0.00 mins
2020-06-17 14:46:42.253260 done after 0.00 mins
2020-06-17 14:46:42.272210 done after 0.00 mins
2020-06-17 14:46:42.354195 done after 0.00 mins
2020-06-17 14:46:42.374442 done after 0.00 mins
2020-06-17 14:46:42.473759 done after 0.00 mins
2020-06-17 14:46:42.563059 done after 0.

2020-06-17 14:46:49.701316 done after 0.00 mins
2020-06-17 14:46:49.766142 done after 0.00 mins
2020-06-17 14:46:49.791075 done after 0.00 mins
2020-06-17 14:46:49.896227 done after 0.00 mins
2020-06-17 14:46:49.960845 done after 0.00 mins
2020-06-17 14:46:50.035648 done after 0.00 mins
2020-06-17 14:46:50.114856 done after 0.00 mins
2020-06-17 14:46:50.199632 done after 0.00 mins
2020-06-17 14:46:50.289390 done after 0.00 mins
2020-06-17 14:46:50.371487 done after 0.00 mins
2020-06-17 14:46:50.468944 done after 0.00 mins
2020-06-17 14:46:50.554715 done after 0.00 mins
2020-06-17 14:46:50.562692 done after 0.00 mins
2020-06-17 14:46:50.631673 done after 0.00 mins
2020-06-17 14:46:50.640728 done after 0.00 mins
2020-06-17 14:46:50.652994 done after 0.00 mins
2020-06-17 14:46:50.720815 done after 0.00 mins
2020-06-17 14:46:50.761872 done after 0.00 mins
2020-06-17 14:46:50.825182 done after 0.00 mins
2020-06-17 14:46:50.845936 done after 0.00 mins
2020-06-17 14:46:50.914800 done after 0.

**Note**

Before parallization. Total runs for estimated 1,000,000 loops took about 3 min

* 2020-06-17 14:41:28.739905 (Beginning!)

* 2020-06-17 14:46:51.676526 (End!)

**Note**

Try to change the above code snippet to run parallelization.

Please do not use jupyter notbook, it won't reflect anything.

If you are like me, curious about strategy's mean-reversion velocity, you might want to read this research report.

[Critical-Line Algorithm for Portfolio Optimization by Bailey and Lopez [2013]](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2197616)

**Hint**

Vectorize iterable then try using "iterrows()" or "itertuples()" if you are using pandas.

In [6]:
print("Total count provided: {0}, total count retrieved: {1}".format(len(jobs), len(df)))
df

Total count provided: 10000, total count retrieved: 10000


[[{'profit': 0.5,
   'loss': 0.5,
   'mean': 1.6614332029031802,
   'std': 0.971519953603021,
   'sharpe': 1.7101380128545143}],
 [{'profit': 0.5,
   'loss': 1.0,
   'mean': 1.690836960196075,
   'std': 0.8553741629691958,
   'sharpe': 1.976722039775904}],
 [{'profit': 0.5,
   'loss': 1.5,
   'mean': 1.6019704310539382,
   'std': 0.665186548309494,
   'sharpe': 2.4083025057033822}],
 [{'profit': 0.5,
   'loss': 2.0,
   'mean': 1.6800109553709877,
   'std': 0.7956590529266577,
   'sharpe': 2.111470923621663}],
 [{'profit': 0.5,
   'loss': 2.5,
   'mean': 1.6284748547761916,
   'std': 0.7573222142544277,
   'sharpe': 2.1503064668179586}],
 [{'profit': 0.5,
   'loss': 3.0,
   'mean': 1.6916628779460277,
   'std': 0.7682696911585473,
   'sharpe': 2.201912814489672}],
 [{'profit': 0.5,
   'loss': 3.5,
   'mean': 1.6814481655854396,
   'std': 0.7648066514547471,
   'sharpe': 2.1985271210535875}],
 [{'profit': 0.5,
   'loss': 4.0,
   'mean': 1.6920193715068808,
   'std': 0.7769855350757856,
 

In [7]:
df1 = rs.opt_tr(profit_range = profit_range,
                  loss_range = loss_range,
                  sigma = sigma,
                  n_run = n_run,
                  max_period = max_period,
                  forecast = forecast, 
                  half_life = half_life,
                  num_threads = 3)

[      profit  loss      mean       std    sharpe
0        0.5   0.5  1.396264  0.984278  1.418566
1        0.5   1.0  1.587675  0.774706  2.049392
2        0.5   1.5  1.712608  0.826244  2.072764
3        0.5   2.0  1.600266  0.662226  2.416496
4        0.5   2.5  1.610815  0.680407  2.367427
...      ...   ...       ...       ...       ...
3329     3.5   5.0  1.380343  4.395407  0.314042
3330     3.5   5.5  2.112414  3.886711  0.543497
3331     3.5   6.0  2.024534  4.154450  0.487317
3332     3.5   6.5  1.968073  4.356944  0.451709
3333     3.5   7.0  3.004285  3.233671  0.929063

[3334 rows x 5 columns]] this out


[      profit  loss      mean       std    sharpe
0        0.5   0.5  1.396264  0.984278  1.418566
1        0.5   1.0  1.587675  0.774706  2.049392
2        0.5   1.5  1.712608  0.826244  2.072764
3        0.5   2.0  1.600266  0.662226  2.416496
4        0.5   2.5  1.610815  0.680407  2.367427
...      ...   ...       ...       ...       ...
3329     3.5   5.0  1.380343  4.395407  0.314042
3330     3.5   5.5  2.112414  3.886711  0.543497
3331     3.5   6.0  2.024534  4.154450  0.487317
3332     3.5   6.5  1.968073  4.356944  0.451709
3333     3.5   7.0  3.004285  3.233671  0.929063

[3334 rows x 5 columns],       profit  loss      mean       std     sharpe
6667     7.0   4.0 -4.659314  0.477871  -9.750155
6668     7.0   4.5 -5.038846  0.450071 -11.195665
6669     7.0   5.0 -5.680721  0.604291  -9.400631
6670     7.0   5.5 -6.030476  0.483595 -12.470101
6671     7.0   6.0 -6.486064  0.876892  -7.396649
...      ...   ...       ...       ...        ...
9995    10.0   8.0 -3.843756  6.173

[      profit  loss      mean       std    sharpe
0        0.5   0.5  1.396264  0.984278  1.418566
1        0.5   1.0  1.587675  0.774706  2.049392
2        0.5   1.5  1.712608  0.826244  2.072764
3        0.5   2.0  1.600266  0.662226  2.416496
4        0.5   2.5  1.610815  0.680407  2.367427
...      ...   ...       ...       ...       ...
3329     3.5   5.0  1.380343  4.395407  0.314042
3330     3.5   5.5  2.112414  3.886711  0.543497
3331     3.5   6.0  2.024534  4.154450  0.487317
3332     3.5   6.5  1.968073  4.356944  0.451709
3333     3.5   7.0  3.004285  3.233671  0.929063

[3334 rows x 5 columns],       profit  loss      mean       std     sharpe
6667     7.0   4.0 -4.659314  0.477871  -9.750155
6668     7.0   4.5 -5.038846  0.450071 -11.195665
6669     7.0   5.0 -5.680721  0.604291  -9.400631
6670     7.0   5.5 -6.030476  0.483595 -12.470101
6671     7.0   6.0 -6.486064  0.876892  -7.396649
...      ...   ...       ...       ...        ...
9995    10.0   8.0 -3.843756  6.173

2020-06-17 14:49:46.331662 100.0% _opt_tr done after 2.88 mins. Remaining 0.0 mins.


In [8]:
df1

,profit,loss,mean,std,sharpe
0,0.5,0.5,1.396264,0.984278,1.418566
1,0.5,1.0,1.587675,0.774706,2.049392
2,0.5,1.5,1.712608,0.826244,2.072764
3,0.5,2.0,1.600266,0.662226,2.416496
4,0.5,2.5,1.610815,0.680407,2.367427
...,...,...,...,...,...
9995,10.0,8.0,-3.843756,6.173427,-0.622629
9996,10.0,8.5,-4.054103,7.336595,-0.552587
9997,10.0,9.0,-4.632491,6.437791,-0.719578
9998,10.0,9.5,-4.404237,7.476366,-0.589088


**Conclusion**

The above was ran parallel with 3 threads:
    
1. 2020-06-17 14:49:16.186467
2. 2020-06-17 14:49:21.045204
3. 2020-06-17 14:49:46.331662

Based on the time provided by python system, it seems that it took less than 40 seconds.

When running large dataset, especially with Pandas parallelization can save time and always vectorize the datasets.

**Note**

If the answer to the above will not tally with the initial run (Scalar Operations), since we are running a Monte-Carlos.